# Chemical-Disease Relation (CDR) Tutorial

In this example, we'll be writing an application to extract *mentions of* **chemical-induced-disease relationships** from Pubmed abstracts, as per the [BioCreative CDR Challenge](http://www.biocreative.org/resources/corpora/biocreative-v-cdr-corpus/).  This tutorial will show off some of the more advanced features of Snorkel, so we'll assume you've followed the Intro tutorial.

Let's start by reloading from the last notebook.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
os.environ['SNORKELDB'] = 'postgres:///cdr-structure-learning-2'

from snorkel import SnorkelSession

session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
test = session.query(ChemicalDisease).filter(ChemicalDisease.split == 2).all()

print 'Training set:\t{0} candidates'.format(len(train))
print 'Dev set:\t{0} candidates'.format(len(dev))
print 'Test set:\t{0} candidates'.format(len(test))    

Training set:	34283 candidates
Dev set:	888 candidates
Test set:	4620 candidates


In [40]:
from snorkel.annotations import load_marginals
train_marginals = load_marginals(session, split=0)

# Training `SparseLogReg`

Instead of LSTM, to start.  First, reloading features:

In [18]:
from snorkel.annotations import FeatureAnnotator
featurizer = FeatureAnnotator()

F_train = featurizer.load_matrix(session, split=0)
F_dev   = featurizer.load_matrix(session, split=1)
F_test  = featurizer.load_matrix(session, split=2)

In [41]:
from snorkel.learning import SparseLogisticRegression
disc_model = SparseLogisticRegression()

In [42]:
from snorkel.learning.utils import MentionScorer
from snorkel.learning import RandomSearch, ListParameter, RangeParameter

# Searching over learning rate
rate_param = RangeParameter('lr', 1e-5, 1e-2, step=1, log_base=10)
l1_param  = RangeParameter('l1_penalty', 1e-4, 1e-2, step=1, log_base=10)
l2_param  = RangeParameter('l2_penalty', 1e-4, 1e-2, step=1, log_base=10)

searcher = RandomSearch(session, disc_model, F_train, train_marginals, [rate_param, l1_param, l2_param], n=10)

Initialized RandomSearch search of size 10. Search space size = 36.


In [43]:
from snorkel.annotations import load_gold_labels
L_gold_dev = load_gold_labels(session, annotator_name='gold', split=1)
L_gold_dev

<888x1 sparse matrix of type '<type 'numpy.float64'>'
	with 888 stored elements in Compressed Sparse Row format>

In [49]:
import numpy as np
np.random.seed(1701)
searcher.fit(F_dev, L_gold_dev, n_epochs=100, rebalance=0.5, print_freq=25)

[1] Testing lr = 1.00e-03, l1_penalty = 1.00e-04, l2_penalty = 1.00e-03
[SparseLR] lr=0.001 l1=0.0001 l2=0.001
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=7524  #epochs=100  batch size=100
[SparseLR] Epoch 0 (1.51s)	Avg. loss=0.693434	NNZ=434414
[SparseLR] Epoch 25 (19.34s)	Avg. loss=0.594222	NNZ=434414
[SparseLR] Epoch 50 (36.32s)	Avg. loss=0.548588	NNZ=434414
[SparseLR] Epoch 75 (53.96s)	Avg. loss=0.519936	NNZ=434416
[SparseLR] Epoch 99 (71.74s)	Avg. loss=0.500509	NNZ=434416
[SparseLR] Training done (71.74s)
[SparseLR] Model saved. To load, use name
		SparseLR_0
[2] Testing lr = 1.00e-05, l1_penalty = 1.00e-02, l2_penalty = 1.00e-04
[SparseLR] lr=1e-05 l1=0.01 l2=0.0001
[SparseLR] Building model
[SparseLR] Training model
[SparseLR] #examples=7524  #epochs=100  batch size=100
[SparseLR] Epoch 0 (1.40s)	Avg. loss=0.693786	NNZ=434415
[SparseLR] Epoch 25 (19.20s)	Avg. loss=0.691744	NNZ=434413
[SparseLR] Epoch 50 (37.06s)	Avg. loss=0.689984	NNZ=434414
[SparseL

,lr,l1_penalty,l2_penalty,Prec.,Rec.,F1
2,0.00100,0.0001,0.0010,0.382743,0.584459,0.462567
0,0.00100,0.0001,0.0010,0.391924,0.557432,0.460251
3,0.00100,0.0100,0.0010,0.377261,0.493243,0.427526
6,0.01000,0.0100,0.0001,0.375000,0.486486,0.423529
5,0.01000,0.0001,0.0010,0.370968,0.466216,0.413174
4,0.00100,0.0100,0.0001,0.349515,0.486486,0.406780
8,0.00001,0.0100,0.0100,0.375740,0.429054,0.400631
9,0.00001,0.0100,0.0100,0.385965,0.371622,0.378657
1,0.00001,0.0100,0.0001,0.348214,0.395270,0.370253
7,0.00010,0.0100,0.0001,0.326019,0.351351,0.338211


### Scoring on the test set

Finally, we'll evaluate our performance on the blind test set of 500 documents. We'll load labels similar to how we did for the development set, and use the `score` function of our extraction model to see how we did.

In [ ]:
from load_external_annotations import load_external_labels
load_external_labels(session, ChemicalDisease, split=2, annotator='gold')

In [15]:
from snorkel.annotations import load_gold_labels
L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_test

<4620x1 sparse matrix of type '<type 'numpy.float64'>'
	with 4620 stored elements in Compressed Sparse Row format>

In [51]:
_, _, _, _ = disc_model.score(session, F_test, L_gold_test, b=0.5)

Scores (Un-adjusted)
Pos. class accuracy: 0.497
Neg. class accuracy: 0.48
Precision            0.316
Recall               0.497
F1                   0.386
----------------------------------------
TP: 748 | FP: 1620 | TN: 1495 | FN: 757



# Part V: Training an LSTM extraction model

In the intro tutorial, we automatically featurized the candidates and trained a linear model over these features. Here, we'll train a more complicated model for relation extraction: an LSTM network. You can read more about LSTMs [here](https://en.wikipedia.org/wiki/Long_short-term_memory) or [here](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). An LSTM is a type of recurrent neural network and automatically generates a numerical representation for the candidate based on the sentence text, so no need for featurizing explicitly as in the intro tutorial. LSTMs take longer to train, and Snorkel doesn't currently support hyperparameter searches for them. We'll train a single model here, but feel free to try out other parameter sets. Just make sure to use the development set - and not the test set - for model selection.

In [ ]:
from snorkel.contrib.learning import reLSTM

lstm = reLSTM()
lstm.train(
    train, train_marginals, lr=0.005, dim=200, n_epochs=30,
    dropout_rate=0.5, rebalance=0.25, print_freq=5
)